# Data viz projecy

### Import useful libraries

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

### Read the data and clean it

Read the data and convert the date format

In [5]:
df=pd.read_csv("UK-HPI-full-file-2023-02.csv"); # load the data
df_reduced=df[['Date', 'RegionName', 'AreaCode', 'AveragePrice']].copy(); # extract the data that refer only to restrict number of columns.
df_reduced['Date'] = pd.to_datetime(df['Date'],dayfirst=True); # By default, the date is imported as object. However, it is better to convert the date column from object to date class

Extract the data matching a specific adate

In [6]:
time_analysis= np.datetime64('2023-01-01'); # in the first phase, will only work with the data for a give date
df_grouped=df_reduced[df_reduced.Date==time_analysis].copy(); # extract the dataset for that given date

Discard any macro-region (e.g. England) from the dataset

In [7]:
regions_to_discard=['England','Scotland','Wales','Northern Ireland']; # We will work with local authorities, so we can discard high level geographical boundaries. Create a list with the elements to discard.
df_final=df_grouped[~df_grouped.RegionName.isin(regions_to_discard)].copy(); # find the rows where the region name is equal a value within the list. The function returns true when the values are met. The ~ operator swaps true and false.
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 228 to 135313
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          401 non-null    datetime64[ns]
 1   RegionName    401 non-null    object        
 2   AreaCode      401 non-null    object        
 3   AveragePrice  401 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 15.7+ KB


Export the data in a json file

In [8]:
df_final['Date'] = df_final['Date'].dt.strftime('%Y-%m-%d')
df_final.to_json(path_or_buf='HPI.json', orient='records')